In [2]:
import numpy as np
import pandas as pd
import sqlite3
import itertools

import os
for dirname, _, filenames in os.walk('.'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

.\Spotify Recommendation.ipynb
.\spotify-data-cleaned.pkl
.\spotify.sqlite
.\.ipynb_checkpoints\Spotify Recommendation-checkpoint.ipynb


In [2]:
con = sqlite3.connect("./spotify.sqlite")
con.text_factory = lambda x: x.decode(encoding='utf-8', errors='ignore')
pd.set_option('display.max_columns', None)

In [3]:
albums_df = pd.read_sql_query("SELECT * from albums", con)
print(albums_df.shape)
albums_df.head()

(4820754, 6)


,id,name,album_group,album_type,release_date,popularity
0,2jKoVlU7VAmExKJ1Jh3w9P,"Alkaholik (feat. Erik Sermon, J Ro & Tash)",,album,954633600000,0
1,4JYUDRtPZuVNi7FAnbHyux,"Flava in Ya Ear Remix (feat. Notorious B.I.G.,...",,single,757382400000,0
2,6YjKAkDYmlasMqYw73iB0w,Bitch Please II,,album,959040000000,0
3,2YlvHjDb4Tyxl4A1IcDhAe,Just Dippin',,compilation,1104537600000,0
4,3UOuBNEin5peSRqdzvlnWM,Still D.R.E.,,album,942710400000,0


In [4]:
artists_df = pd.read_sql_query("SELECT * from artists", con)
print(artists_df.shape)
artists_df.head()

(1066031, 4)


,name,id,popularity,followers
0,Xzibit,4tujQJicOnuZRLiBFdp3Ou,69,1193665
1,Erick Sermon,2VX0o9LDIVmKIgpnwdJpOJ,54,142007
2,J. Ro,3iBOsmwGzRKyR0vs2I61xP,45,158
3,Tash,22qf8cJRzBjIWb2Jc4JeOr,48,3421
4,Craig Mack,4akj4uteQQrrGxhX9Rjuyf,55,161966


In [8]:
audiofeatures_df = pd.read_sql_query("SELECT * from audio_features", con)
audiofeatures_df.rename(columns={'id': 'audio_feature_id'}, inplace=True)
print(audiofeatures_df.shape)
audiofeatures_df.head()

(8740043, 15)


,audio_feature_id,acousticness,analysis_url,danceability,duration,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence
0,2jKoVlU7VAmExKJ1Jh3w9P,0.1800,https://api.spotify.com/v1/audio-analysis/2jKo...,0.893,219160,0.514,0.000000,11,0.0596,-5.080,1,0.283,95.848000,4,0.787
1,4JYUDRtPZuVNi7FAnbHyux,0.2720,https://api.spotify.com/v1/audio-analysis/4JYU...,0.520,302013,0.847,0.000000,9,0.3250,-5.300,1,0.427,177.371002,4,0.799
2,6YjKAkDYmlasMqYw73iB0w,0.0783,https://api.spotify.com/v1/audio-analysis/6YjK...,0.918,288200,0.586,0.000000,1,0.1450,-2.890,1,0.133,95.516998,4,0.779
3,2YlvHjDb4Tyxl4A1IcDhAe,0.5840,https://api.spotify.com/v1/audio-analysis/2Ylv...,0.877,243013,0.681,0.000000,1,0.1190,-6.277,0,0.259,94.834999,4,0.839
4,3UOuBNEin5peSRqdzvlnWM,0.1700,https://api.spotify.com/v1/audio-analysis/3UOu...,0.814,270667,0.781,0.000518,11,0.0520,-3.330,1,0.233,93.445000,4,0.536


In [9]:
genres_df = pd.read_sql_query("SELECT * from genres", con)
print(genres_df.shape)
genres_df.head()

(5489, 1)


,id
0,detroit hip hop
1,g funk
2,gangster rap
3,hardcore hip hop
4,hip hop


In [10]:
rAlbumsArtists_df = pd.read_sql_query("SELECT * from r_albums_artists", con)
print(rAlbumsArtists_df.shape)
rAlbumsArtists_df.head()

(921486, 2)


,album_id,artist_id
0,6os2Mv58OYnQClPf7B9E1s,2HS2wQTJXpA65XWOKlAVxk
1,5XXN1tFQg7D7U1NSVh5fjf,3VBpsrUi2vV7Uj87ONHu7Z
2,63wFVCPZxdaFL9WKXbIQ5m,1kC8lmaSblf7GyK915496r
3,0z7p1tHVJ6s5wxAdj9iJ6e,2bTFaANevDESpT5eGWsZ7a
4,4afYJvrsp90d3V2COlLHVE,2maQMqxNnlRrBrS1oAsrX9


In [12]:
rAlbumsTracks_df = pd.read_sql_query("SELECT * from r_albums_tracks", con)
print(rAlbumsTracks_df.shape)
rAlbumsTracks_df.head()

(9900173, 2)


,album_id,track_id
0,6os2Mv58OYnQClPf7B9E1s,3HnrHGLE9u2MjHtdobfWl9
1,6os2Mv58OYnQClPf7B9E1s,4lDjkpUrpWlMFofIpzuExK
2,6os2Mv58OYnQClPf7B9E1s,4RDT3ftT3V0IJVmW0TNgof
3,6os2Mv58OYnQClPf7B9E1s,1DfwOz5apF0w2d8hccSkhH
4,6os2Mv58OYnQClPf7B9E1s,46JY9OYPi4hJ7oLIS0FZt0


In [14]:
rArtistGenre_df = pd.read_sql_query("SELECT * from r_artist_genre", con)
rArtistGenre_df = rArtistGenre_df.groupby('artist_id')['genre_id'].apply(list).reset_index()
print(rArtistGenre_df.shape)
rArtistGenre_df.head()

(326123, 2)


,artist_id,genre_id
0,0001ZVMPt41Vwzt1zsmuzp,[opm]
1,0001wHqxbF2YYRQxGdbyER,[progressive psytrance]
2,00045gNg7mLEf9UY9yhD0t,[dutch hip hop]
3,0009v6e5hmLdAqSFTLcKry,[classic russian pop]
4,000Dq0VqTZpxOP6jQMscVL,"[baton rouge rap, deep southern trap]"


In [15]:
rTrackArtist_df = pd.read_sql_query("SELECT * from r_track_artist", con)
rTrackArtist_df['track_id'] = rTrackArtist_df['track_id'].astype('|S')
print(rTrackArtist_df.shape)
rTrackArtist_df.head()

(11840402, 2)


,track_id,artist_id
0,b'2jKoVlU7VAmExKJ1Jh3w9P',4tujQJicOnuZRLiBFdp3Ou
1,b'2jKoVlU7VAmExKJ1Jh3w9P',2VX0o9LDIVmKIgpnwdJpOJ
2,b'2jKoVlU7VAmExKJ1Jh3w9P',3iBOsmwGzRKyR0vs2I61xP
3,b'2jKoVlU7VAmExKJ1Jh3w9P',22qf8cJRzBjIWb2Jc4JeOr
4,b'4JYUDRtPZuVNi7FAnbHyux',4akj4uteQQrrGxhX9Rjuyf


In [16]:
tracks_df = pd.read_sql_query("SELECT * from tracks", con)
print(tracks_df.shape)
tracks_df.head()

(8741672, 10)


,id,disc_number,duration,explicit,audio_feature_id,name,preview_url,track_number,popularity,is_playable
0,1dizvxctg9dHEyaYTFufVi,1,275893,1,1dizvxctg9dHEyaYTFufVi,Gz And Hustlas (feat. Nancy Fletcher),,12,0,NaN
1,2g8HN35AnVGIk7B8yMucww,1,252746,1,2g8HN35AnVGIk7B8yMucww,Big Poppa - 2005 Remaster,https://p.scdn.co/mp3-preview/770e023eb0318270...,13,77,NaN
2,49pnyECzcMGCKAqxfTB4JZ,3,315080,0,49pnyECzcMGCKAqxfTB4JZ,"You Were Born - Early Version Of ""One Of The T...",,6,8,1.0
3,4E5IFAXCob6QqZaJMTw5YN,1,240800,1,4E5IFAXCob6QqZaJMTw5YN,Poppin' Them Thangs,https://p.scdn.co/mp3-preview/f3b556ced9657f89...,2,70,NaN
4,1gSt2UlC7mtRtJIc5zqKWn,2,203666,0,1gSt2UlC7mtRtJIc5zqKWn,"It's Hard To Say ""I Do"", When I Don't",,2,50,NaN


In [17]:
merged_tracks = pd.merge(rTrackArtist_df, rArtistGenre_df, left_on='artist_id', right_on='artist_id')
merged_tracks = pd.merge(merged_tracks, artists_df, left_on='artist_id', right_on='id')
merged_tracks.drop(['id', 'popularity', 'followers'],axis=1,inplace=True)
merged_tracks.rename(columns={'name': 'artist_name'}, inplace=True)
merged_tracks.head()

,track_id,artist_id,genre_id,artist_name
0,b'2jKoVlU7VAmExKJ1Jh3w9P',4tujQJicOnuZRLiBFdp3Ou,"[detroit hip hop, g funk, gangster rap, hardco...",Xzibit
1,b'6YjKAkDYmlasMqYw73iB0w',4tujQJicOnuZRLiBFdp3Ou,"[detroit hip hop, g funk, gangster rap, hardco...",Xzibit
2,b'1ktyqk3vVMB3zg17NjttnX',4tujQJicOnuZRLiBFdp3Ou,"[detroit hip hop, g funk, gangster rap, hardco...",Xzibit
3,b'0GQQzmMNA7JE4tOdcSFh1C',4tujQJicOnuZRLiBFdp3Ou,"[detroit hip hop, g funk, gangster rap, hardco...",Xzibit
4,b'1SWVDBtw6h3tm9OehOkDhv',4tujQJicOnuZRLiBFdp3Ou,"[detroit hip hop, g funk, gangster rap, hardco...",Xzibit


In [18]:
merged_tracks['track_id'] = merged_tracks['track_id'].str.decode("utf-8")
merged_tracks.head()

,track_id,artist_id,genre_id,artist_name
0,2jKoVlU7VAmExKJ1Jh3w9P,4tujQJicOnuZRLiBFdp3Ou,"[detroit hip hop, g funk, gangster rap, hardco...",Xzibit
1,6YjKAkDYmlasMqYw73iB0w,4tujQJicOnuZRLiBFdp3Ou,"[detroit hip hop, g funk, gangster rap, hardco...",Xzibit
2,1ktyqk3vVMB3zg17NjttnX,4tujQJicOnuZRLiBFdp3Ou,"[detroit hip hop, g funk, gangster rap, hardco...",Xzibit
3,0GQQzmMNA7JE4tOdcSFh1C,4tujQJicOnuZRLiBFdp3Ou,"[detroit hip hop, g funk, gangster rap, hardco...",Xzibit
4,1SWVDBtw6h3tm9OehOkDhv,4tujQJicOnuZRLiBFdp3Ou,"[detroit hip hop, g funk, gangster rap, hardco...",Xzibit


In [26]:
grouped_tracks = merged_tracks.groupby('track_id').agg({
    'artist_id': list,
    'genre_id': lambda x: list(set(itertools.chain(*x))),
    'artist_name': list
}).reset_index()

grouped_tracks.head()

,track_id,artist_id,genre_id,artist_name
0,0000QBRGPosiFRXKmMYnsO,[05JwisTou63l1k92FQEyhT],[cha-cha-cha],[Sexteto La Playa]
1,0000gBWfr2zIFzE5tDzxca,[0Tgtl5beIMahbtIzV5jBXw],[ukrainian metal],[Rattenfänger]
2,0000uJA4xCdxThagdLkkLR,[5kEVfWQGTw0rIDO2Jqq1ww],"[neo-traditional bluegrass, progressive bluegr...",[Cherryholmes]
3,0001Lyv0YTjkZSqzT4WkLy,[1YPYaJyUobMi0eABhZo92N],"[outsider, gbvfi, dunedin sound, post-punk, lo...",[Half Japanese]
4,0001QZSdENvrMx6cZXZJdo,[0pNaVvqSvldpJl7pHpNoM9],"[gbvfi, triangle indie, alternative rock, alte...",[Archers Of Loaf]


In [59]:
merged = pd.merge(tracks_df, grouped_tracks, left_on='id', right_on='track_id')
merged.drop(['preview_url', 'track_id', 'is_playable'], axis=1, inplace=True)
merged.head()

,id,disc_number,duration,explicit,audio_feature_id,name,track_number,popularity,artist_id,genre_id,artist_name
0,1dizvxctg9dHEyaYTFufVi,1,275893,1,1dizvxctg9dHEyaYTFufVi,Gz And Hustlas (feat. Nancy Fletcher),12,0,[7hJcb9fa4alzcOq3EaNPoG],"[g funk, hip hop, west coast rap, pop rap, rap...",[Snoop Dogg]
1,2g8HN35AnVGIk7B8yMucww,1,252746,1,2g8HN35AnVGIk7B8yMucww,Big Poppa - 2005 Remaster,13,77,[5me0Irg2ANcsgc93uaYrpb],"[hip hop, east coast hip hop, hardcore hip hop...",[The Notorious B.I.G.]
2,49pnyECzcMGCKAqxfTB4JZ,3,315080,0,49pnyECzcMGCKAqxfTB4JZ,"You Were Born - Early Version Of ""One Of The T...",6,8,[0qLNsNKm8bQcMoRFkR8Hmh],"[permanent wave, pop rock, rock, new wave, mad...",[James]
3,4E5IFAXCob6QqZaJMTw5YN,1,240800,1,4E5IFAXCob6QqZaJMTw5YN,Poppin' Them Thangs,2,70,[6evKD5JWJON3qPBJtUEmtY],"[hip hop, southern hip hop, trap, pop rap, har...",[G-Unit]
4,1gSt2UlC7mtRtJIc5zqKWn,2,203666,0,1gSt2UlC7mtRtJIc5zqKWn,"It's Hard To Say ""I Do"", When I Don't",2,50,[4UXqAaa6dQYAk18Lv7PEgX],"[pop punk, emo, modern rock]",[Fall Out Boy]


In [60]:
merged = pd.merge(merged, audiofeatures_df, left_on='audio_feature_id', right_on='audio_feature_id')
merged.head()

,id,disc_number,duration_x,explicit,audio_feature_id,name,track_number,popularity,artist_id,genre_id,artist_name,acousticness,analysis_url,danceability,duration_y,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence
0,1dizvxctg9dHEyaYTFufVi,1,275893,1,1dizvxctg9dHEyaYTFufVi,Gz And Hustlas (feat. Nancy Fletcher),12,0,[7hJcb9fa4alzcOq3EaNPoG],"[g funk, hip hop, west coast rap, pop rap, rap...",[Snoop Dogg],0.1640,https://api.spotify.com/v1/audio-analysis/1diz...,0.652,275893,0.814,0.000,1,0.3600,-4.901,1,0.3100,91.888000,4,0.788
1,2g8HN35AnVGIk7B8yMucww,1,252746,1,2g8HN35AnVGIk7B8yMucww,Big Poppa - 2005 Remaster,13,77,[5me0Irg2ANcsgc93uaYrpb],"[hip hop, east coast hip hop, hardcore hip hop...",[The Notorious B.I.G.],0.4300,https://api.spotify.com/v1/audio-analysis/2g8H...,0.780,252747,0.575,0.000,9,0.1430,-7.247,0,0.2730,84.491997,4,0.773
2,49pnyECzcMGCKAqxfTB4JZ,3,315080,0,49pnyECzcMGCKAqxfTB4JZ,"You Were Born - Early Version Of ""One Of The T...",6,8,[0qLNsNKm8bQcMoRFkR8Hmh],"[permanent wave, pop rock, rock, new wave, mad...",[James],0.3310,https://api.spotify.com/v1/audio-analysis/49pn...,0.624,315080,0.564,0.293,7,0.2560,-8.928,1,0.0297,111.960999,4,0.431
3,4E5IFAXCob6QqZaJMTw5YN,1,240800,1,4E5IFAXCob6QqZaJMTw5YN,Poppin' Them Thangs,2,70,[6evKD5JWJON3qPBJtUEmtY],"[hip hop, southern hip hop, trap, pop rap, har...",[G-Unit],0.2630,https://api.spotify.com/v1/audio-analysis/4E5I...,0.769,240800,0.818,0.000,6,0.0787,-1.385,0,0.1920,88.016998,4,0.918
4,1gSt2UlC7mtRtJIc5zqKWn,2,203666,0,1gSt2UlC7mtRtJIc5zqKWn,"It's Hard To Say ""I Do"", When I Don't",2,50,[4UXqAaa6dQYAk18Lv7PEgX],"[pop punk, emo, modern rock]",[Fall Out Boy],0.0196,https://api.spotify.com/v1/audio-analysis/1gSt...,0.420,203667,0.949,0.000,10,0.2840,-3.902,1,0.0823,163.945007,4,0.594


In [61]:
merged.drop(['analysis_url', 'duration_y', 'disc_number', 'track_number'], axis=1, inplace=True)
merged.rename(columns={'duration_x': 'duration'}, inplace=True)
merged.head()

,id,duration,explicit,audio_feature_id,name,popularity,artist_id,genre_id,artist_name,acousticness,danceability,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence
0,1dizvxctg9dHEyaYTFufVi,275893,1,1dizvxctg9dHEyaYTFufVi,Gz And Hustlas (feat. Nancy Fletcher),0,[7hJcb9fa4alzcOq3EaNPoG],"[g funk, hip hop, west coast rap, pop rap, rap...",[Snoop Dogg],0.1640,0.652,0.814,0.000,1,0.3600,-4.901,1,0.3100,91.888000,4,0.788
1,2g8HN35AnVGIk7B8yMucww,252746,1,2g8HN35AnVGIk7B8yMucww,Big Poppa - 2005 Remaster,77,[5me0Irg2ANcsgc93uaYrpb],"[hip hop, east coast hip hop, hardcore hip hop...",[The Notorious B.I.G.],0.4300,0.780,0.575,0.000,9,0.1430,-7.247,0,0.2730,84.491997,4,0.773
2,49pnyECzcMGCKAqxfTB4JZ,315080,0,49pnyECzcMGCKAqxfTB4JZ,"You Were Born - Early Version Of ""One Of The T...",8,[0qLNsNKm8bQcMoRFkR8Hmh],"[permanent wave, pop rock, rock, new wave, mad...",[James],0.3310,0.624,0.564,0.293,7,0.2560,-8.928,1,0.0297,111.960999,4,0.431
3,4E5IFAXCob6QqZaJMTw5YN,240800,1,4E5IFAXCob6QqZaJMTw5YN,Poppin' Them Thangs,70,[6evKD5JWJON3qPBJtUEmtY],"[hip hop, southern hip hop, trap, pop rap, har...",[G-Unit],0.2630,0.769,0.818,0.000,6,0.0787,-1.385,0,0.1920,88.016998,4,0.918
4,1gSt2UlC7mtRtJIc5zqKWn,203666,0,1gSt2UlC7mtRtJIc5zqKWn,"It's Hard To Say ""I Do"", When I Don't",50,[4UXqAaa6dQYAk18Lv7PEgX],"[pop punk, emo, modern rock]",[Fall Out Boy],0.0196,0.420,0.949,0.000,10,0.2840,-3.902,1,0.0823,163.945007,4,0.594


In [62]:
albums_df['release_date'] = albums_df['release_date'].clip(lower=-6000000000000)
albums_df['release_datetime'] = pd.to_datetime(albums_df['release_date'], origin='unix', unit='ms')
albums_df['year'] = albums_df['release_datetime'].dt.year
albums_df.head()

KeyError: 'release_date'

In [63]:
albums_df.drop(['name', 'album_group', 'album_type', 'release_date', 'release_date', 'popularity', 'release_datetime'], axis=1, inplace=True)
albums_df.rename(columns={'id': 'album_id'}, inplace=True)
albums_df.head()

KeyError: "['name' 'album_group' 'album_type' 'release_date' 'release_date'\n 'popularity' 'release_datetime'] not found in axis"

In [64]:
merged_year = pd.merge(rAlbumsTracks_df, albums_df, left_on='album_id', right_on='album_id')
merged_year.head()

,album_id,track_id,year
0,6os2Mv58OYnQClPf7B9E1s,3HnrHGLE9u2MjHtdobfWl9,1920
1,6os2Mv58OYnQClPf7B9E1s,4lDjkpUrpWlMFofIpzuExK,1920
2,6os2Mv58OYnQClPf7B9E1s,4RDT3ftT3V0IJVmW0TNgof,1920
3,6os2Mv58OYnQClPf7B9E1s,1DfwOz5apF0w2d8hccSkhH,1920
4,6os2Mv58OYnQClPf7B9E1s,46JY9OYPi4hJ7oLIS0FZt0,1920


In [68]:
merged2 = pd.merge(merged, merged_year, left_on='id', right_on='track_id')
merged2.head()

,id,duration,explicit,audio_feature_id,name,popularity,artist_id,genre_id,artist_name,acousticness,danceability,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence,album_id,track_id,year
0,2g8HN35AnVGIk7B8yMucww,252746,1,2g8HN35AnVGIk7B8yMucww,Big Poppa - 2005 Remaster,77,[5me0Irg2ANcsgc93uaYrpb],"[hip hop, east coast hip hop, hardcore hip hop...",[The Notorious B.I.G.],0.4300,0.780,0.575,0.000000,9,0.1430,-7.247,0,0.2730,84.491997,4,0.773,2HTbQ0RHwukKVXAlTmCZP2,2g8HN35AnVGIk7B8yMucww,1994
1,4E5IFAXCob6QqZaJMTw5YN,240800,1,4E5IFAXCob6QqZaJMTw5YN,Poppin' Them Thangs,70,[6evKD5JWJON3qPBJtUEmtY],"[hip hop, southern hip hop, trap, pop rap, har...",[G-Unit],0.2630,0.769,0.818,0.000000,6,0.0787,-1.385,0,0.1920,88.016998,4,0.918,0BolFrIcCXXppUK50ETvgy,4E5IFAXCob6QqZaJMTw5YN,2003
2,1gSt2UlC7mtRtJIc5zqKWn,203666,0,1gSt2UlC7mtRtJIc5zqKWn,"It's Hard To Say ""I Do"", When I Don't",50,[4UXqAaa6dQYAk18Lv7PEgX],"[pop punk, emo, modern rock]",[Fall Out Boy],0.0196,0.420,0.949,0.000000,10,0.2840,-3.902,1,0.0823,163.945007,4,0.594,0hHopYqXhuvYSHtVyrcb1g,1gSt2UlC7mtRtJIc5zqKWn,2007
3,67IvfvAMYQzJEeHopvwMMW,173159,1,67IvfvAMYQzJEeHopvwMMW,Simon Says,62,[5DKuVtlpDH0agZQUFDy8O7],"[turntablism, hip hop, conscious hip hop, east...",[Pharoahe Monch],0.1700,0.664,0.916,0.000017,1,0.1640,-3.888,1,0.4730,93.134003,4,0.559,2hKCg2JLGtjATrYvpQedT5,67IvfvAMYQzJEeHopvwMMW,1999
4,2I9foKseoFQh07p6sD2voE,256226,1,2I9foKseoFQh07p6sD2voE,Many Men (Wish Death),68,[3q7HBObVc0L8jNeTe5Gofh],"[hip hop, east coast hip hop, pop rap, queens ...",[50 Cent],0.0153,0.653,0.826,0.001460,6,0.1310,-2.930,0,0.3540,108.519997,5,0.416,4ycNE7y1rp5215g1kkqk1P,2I9foKseoFQh07p6sD2voE,2003


In [69]:
merged2.drop(['audio_feature_id','artist_id', 'explicit', 'album_id', 'track_id'], axis=1, inplace=True)
merged2.head()

,id,duration,name,popularity,genre_id,artist_name,acousticness,danceability,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence,year
0,2g8HN35AnVGIk7B8yMucww,252746,Big Poppa - 2005 Remaster,77,"[hip hop, east coast hip hop, hardcore hip hop...",[The Notorious B.I.G.],0.4300,0.780,0.575,0.000000,9,0.1430,-7.247,0,0.2730,84.491997,4,0.773,1994
1,4E5IFAXCob6QqZaJMTw5YN,240800,Poppin' Them Thangs,70,"[hip hop, southern hip hop, trap, pop rap, har...",[G-Unit],0.2630,0.769,0.818,0.000000,6,0.0787,-1.385,0,0.1920,88.016998,4,0.918,2003
2,1gSt2UlC7mtRtJIc5zqKWn,203666,"It's Hard To Say ""I Do"", When I Don't",50,"[pop punk, emo, modern rock]",[Fall Out Boy],0.0196,0.420,0.949,0.000000,10,0.2840,-3.902,1,0.0823,163.945007,4,0.594,2007
3,67IvfvAMYQzJEeHopvwMMW,173159,Simon Says,62,"[turntablism, hip hop, conscious hip hop, east...",[Pharoahe Monch],0.1700,0.664,0.916,0.000017,1,0.1640,-3.888,1,0.4730,93.134003,4,0.559,1999
4,2I9foKseoFQh07p6sD2voE,256226,Many Men (Wish Death),68,"[hip hop, east coast hip hop, pop rap, queens ...",[50 Cent],0.0153,0.653,0.826,0.001460,6,0.1310,-2.930,0,0.3540,108.519997,5,0.416,2003


In [1]:
merged2.to_pickle('./spotify-data-cleaned.pkl')

NameError: name 'merged2' is not defined

In [3]:
spotify_data = pd.read_pickle('./spotify-data-cleaned.pkl')
spotify_data.head()

,id,duration,name,popularity,genre_id,artist_name,acousticness,danceability,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence,year
0,2g8HN35AnVGIk7B8yMucww,252746,Big Poppa - 2005 Remaster,77,"[hip hop, east coast hip hop, hardcore hip hop...",[The Notorious B.I.G.],0.4300,0.780,0.575,0.000000,9,0.1430,-7.247,0,0.2730,84.491997,4,0.773,1994
1,4E5IFAXCob6QqZaJMTw5YN,240800,Poppin' Them Thangs,70,"[hip hop, southern hip hop, trap, pop rap, har...",[G-Unit],0.2630,0.769,0.818,0.000000,6,0.0787,-1.385,0,0.1920,88.016998,4,0.918,2003
2,1gSt2UlC7mtRtJIc5zqKWn,203666,"It's Hard To Say ""I Do"", When I Don't",50,"[pop punk, emo, modern rock]",[Fall Out Boy],0.0196,0.420,0.949,0.000000,10,0.2840,-3.902,1,0.0823,163.945007,4,0.594,2007
3,67IvfvAMYQzJEeHopvwMMW,173159,Simon Says,62,"[turntablism, hip hop, conscious hip hop, east...",[Pharoahe Monch],0.1700,0.664,0.916,0.000017,1,0.1640,-3.888,1,0.4730,93.134003,4,0.559,1999
4,2I9foKseoFQh07p6sD2voE,256226,Many Men (Wish Death),68,"[hip hop, east coast hip hop, pop rap, queens ...",[50 Cent],0.0153,0.653,0.826,0.001460,6,0.1310,-2.930,0,0.3540,108.519997,5,0.416,2003


In [5]:
#simple function to create OHE features
#this gets passed later on
def ohe_prep(df, column, new_name): 
    """ 
    Create One Hot Encoded features of a specific column

    Parameters: 
        df (pandas dataframe): Spotify Dataframe
        column (str): Column to be processed
        new_name (str): new column name to be used
        
    Returns: 
        tf_df: One hot encoded features 
    """
    
    tf_df = pd.get_dummies(df[column])
    feature_names = tf_df.columns
    tf_df.columns = [new_name + "|" + str(i) for i in feature_names]
    tf_df.reset_index(drop = True, inplace = True)    
    return tf_df

In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import MinMaxScaler

#function to build entire feature set
def create_feature_set(df, float_cols):
    """ 
    Process spotify df to create a final set of features that will be used to generate recommendations

    Parameters: 
        df (pandas dataframe): Spotify Dataframe
        float_cols (list(str)): List of float columns that will be scaled 
        
    Returns: 
        final: final set of features 
    """
    
    #tfidf genre lists
    tfidf = TfidfVectorizer()
    tfidf_matrix =  tfidf.fit_transform(df['consolidates_genre_lists'].apply(lambda x: " ".join(x)))
    genre_df = pd.DataFrame(tfidf_matrix.toarray())
    genre_df.columns = ['genre' + "|" + i for i in tfidf.get_feature_names()]
    genre_df.reset_index(drop = True, inplace=True)

    #explicity_ohe = ohe_prep(df, 'explicit','exp')    
    year_ohe = ohe_prep(df, 'year','year') * 0.5
    popularity_ohe = ohe_prep(df, 'popularity_red','pop') * 0.15

    #scale float columns
    floats = df[float_cols].reset_index(drop = True)
    scaler = MinMaxScaler()
    floats_scaled = pd.DataFrame(scaler.fit_transform(floats), columns = floats.columns) * 0.2

    #concanenate all features
    final = pd.concat([genre_df, floats_scaled, popularity_ohe, year_ohe], axis = 1)
     
    #add song id
    final['id']=df['id'].values
    
    return final